In [1]:
!pip install pillow-heif
import torch
import requests
from transformers import SamModel, SamProcessor, CLIPProcessor, CLIPModel
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import os
import pandas as pd

#for authing drive, and file hadnling on drive
import gspread
from google.auth import default
from google.colab import auth

from googleapiclient.http import MediaIoBaseDownload
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#for opening images, and heic files
from PIL import Image
from pillow_heif import register_heif_opener


from sklearn.model_selection import train_test_split
drive.mount('/content/drive')
device = "cuda" if torch.cuda.is_available() else "cpu"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.0 MB/s eta 0:00:00
Mounted at /content/drive


google colab accessing google sheets example notebook: https://colab.research.google.com/notebooks/io.ipynb#scrollTo=vz-jH8T_Uk2c

In [2]:
img_list = []

#need this since some files does not seem to be saved correctly as jpg but as heic
register_heif_opener()

for anImg in os.listdir('/content/drive/MyDrive/Stave Project/Developers/trainTestExampleSet/test'):
  if anImg.endswith('.jpg'):
    img_list.append(Image.open(os.path.join('/content/drive/MyDrive/Stave Project/Developers/trainTestExampleSet/test', anImg)))

staves_count_list = []
with open('/content/drive/MyDrive/Stave Project/Developers/trainTestExampleSet/test/staves_count.txt', 'r') as f:
  for line in f:
    staves_count_list.append(int(line.split(':')[-1].strip()))

print(staves_count_list)

[624, 618, 608, 664, 714, 648, 721, 678, 628, 634, 667, 641, 650, 663, 843, 678, 649, 657, 689, 680, 636, 746, 725, 637, 685, 650, 658, 640, 654, 654, 734]


In [3]:
#laod from huggingface
model_clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor_clip = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [4]:
#note: change the name of these 2 variable below in both lines 10 and 2 in the cell below to get the correct predicted class
clip_numerical_classes = [str(i)+" staves" for i in range(100, 1000)]
clip_object_classes = ["staves", "car", "human", "warehouse"]

In [5]:
#get text-img embedding stuff
predicted_categories=[]
for anImg, cur_stave_count in zip(img_list, staves_count_list):
  inputs = processor_clip(text=clip_numerical_classes, images=anImg, return_tensors="pt", padding=True)
  inputs = {key: value.to(device) for key, value in inputs.items()}
  with torch.no_grad():
      outputs = model_clip(**inputs)
  #get logies
  logits_per_image = outputs.logits_per_image
  #need this for extracting top classes
  predicted_indices = torch.argmax(logits_per_image, dim=1)
  cur_prediction=[clip_numerical_classes[idx] for idx in predicted_indices]
  predicted_categories.append(cur_prediction[0])

In [10]:
clip_results_df = pd.DataFrame({
    "img_names": [os.path.basename(anImg.filename) for anImg in img_list],
    "predicted_category": predicted_categories,
    "correct_stave_count": staves_count_list,

})

clip_results_df['predicted_category_asInt'] = clip_results_df['predicted_category'].str.replace(' staves', '').astype(int)
clip_results_df['accuracy'] = (abs(clip_results_df['predicted_category_asInt'] - clip_results_df['correct_stave_count'])) / clip_results_df['correct_stave_count']
clip_results_df['accuracy'] = 1 - clip_results_df['accuracy']
clip_results_df['MSE'] = (clip_results_df['predicted_category_asInt'] - clip_results_df['correct_stave_count'])**2
clip_results_df

,img_names,predicted_category,correct_stave_count,predicted_category_asInt,accuracy,MSE
0,17296241388511163724737653705721 - Brandon Wag...,205 staves,624,205,0.328526,175561
1,IMG_3179 - Patrick Portell.jpg,580 staves,618,580,0.938511,1444
2,17337595927675172800426821282132 - Brandon Wag...,485 staves,608,485,0.797697,15129
3,IMG_3106 - Patrick Portell.jpg,195 staves,664,195,0.293675,219961
4,20241112_204716 - jast bentahm.jpg,450 staves,714,450,0.630252,69696
5,17339267265701609710443668351905 - Brandon Wag...,195 staves,648,195,0.300926,205209
6,17339393925198142125951619782902 - Brandon Wag...,195 staves,721,195,0.270458,276676
7,17343749120519004743771338510258 - Brandon Wag...,195 staves,678,195,0.287611,233289
8,17301268802994661969305683329586 - Brandon Wag...,205 staves,628,205,0.326433,178929
9,17307358942925383434238657492980 - Brandon Wag...,195 staves,634,195,0.307571,192721


In [12]:
print(f"avg accuracy: {clip_results_df['accuracy'].mean()}")
print(f"avg MSE: {clip_results_df['MSE'].mean()}")

avg accuracy: 0.5508313816227698
avg MSE: 119535.19354838709
